In [ ]:
import torch
import torch.nn as nn
from torchvision import models
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
from sklearn.metrics import classification_report

from matplotlib import pyplot as plt
from PIL import Image
import torchvision

import numpy as np
import tqdm

In [ ]:
unique_labels = ['White', 'Black', 'Asian', 'Indian']

In [ ]:
class ResNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.resnet = models.resnet18(weights=None)
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_features, len(unique_labels))

    def forward(self, x):
        return self.resnet(x)

In [ ]:
net = ResNet()
lr = 0.001
nb_epochs = 30
batch_size = 128
seed = 17
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [ ]:
!wget "https://filebin.net/x73srv8j6rcnvisn/utk_races_seed17.zip"
!unzip utk_races_seed17.zip -d ./data/

Streaming output truncated to the last 5000 lines.
  inflating: ./data/utk_races_seed17/val/2/5_1_2_20161219203429011.jpg.chip.jpg  
  inflating: ./data/utk_races_seed17/val/2/30_0_2_20170116192359839.jpg.chip.jpg  
  inflating: ./data/utk_races_seed17/val/2/8_1_2_20161219163614671.jpg.chip.jpg  
  inflating: ./data/utk_races_seed17/val/2/26_1_2_20170116184821778.jpg.chip.jpg  
  inflating: ./data/utk_races_seed17/val/2/2_0_2_20161219142243497.jpg.chip.jpg  
  inflating: ./data/utk_races_seed17/val/2/26_1_2_20170116161848983.jpg.chip.jpg  
  inflating: ./data/utk_races_seed17/val/2/26_0_2_20170120134050439.jpg.chip.jpg  
  inflating: ./data/utk_races_seed17/val/2/48_0_2_20170117160456200.jpg.chip.jpg  
  inflating: ./data/utk_races_seed17/val/2/4_0_2_20170110225135002.jpg.chip.jpg  
  inflating: ./data/utk_races_seed17/val/2/25_1_2_20170116162822749.jpg.chip.jpg  
  inflating: ./data/utk_races_seed17/val/2/35_0_2_20170116181329511.jpg.chip.jpg  
  inflating: ./data/utk_races_seed17/val

In [ ]:
#%% Define transformations
train_transform = transforms.Compose([
    transforms.Resize(size=128),
    transforms.RandomCrop(104),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize( [0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
test_transform = transforms.Compose([
    transforms.Resize(size=128),
    transforms.CenterCrop(size=104),
    transforms.ToTensor(),
    transforms.Normalize( [0.5, 0.5, 0.5],[0.5, 0.5, 0.5])
])

#%% Load datasets and apply transformations
train_dataset = ImageFolder('data/utk_races_seed{}/train/'.format(seed), transform=train_transform)
val_dataset = ImageFolder('data/utk_races_seed{}/val/'.format(seed), transform=test_transform)
test_dataset = ImageFolder('data/utk_races_seed{}/test/'.format(seed), transform=test_transform)

#%% Create dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
inv_normalize = transforms.Compose([ transforms.Normalize(mean = [ 0., 0., 0. ],
                                                     std = [ 1/0.5, 1/0.5, 1/0.5 ]),
                                transforms.Normalize(mean = [ -0.5, -0.5, -0.5 ],
                                                     std = [ 1., 1., 1. ]),
                               ])

In [ ]:
# visualize the transformed example
grid = torchvision.utils.make_grid(test_dataset[i][0], nrow=1)
transforms.ToPILImage()(grid)

NameError: name 'i' is not defined

In [ ]:
# visualize the original
grid = torchvision.utils.make_grid(inv_normalize(test_dataset[i][0]), nrow=1)
transforms.ToPILImage()(grid)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cuda":
    net = net.cuda()

In [ ]:
!wget "https://filebin.net/bnu2aev5w47bcsg9/utk_races_model_checkpoint.pth"

In [ ]:
net = ResNet()
net.load_state_dict(torch.load('./outputs/best_model.pth')['model_state_dict'])
device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cuda":
    net = net.cuda()
print("loaded best model")

In [ ]:
def fgsm_attack(x, x_grad, eps=0.3):
     sign_x_grad = torch.sign(x_grad)
     adv_x = x_eps * sign_x_grad
    return adv_x, eps*sign_x_grad

In [ ]:
net.eval()
ys = [] # all ground-truth
y_preds = [] # all clean predictions
y_adv_preds = [] # all adversarial predictions
x_advs = [] # all adversarial examples
perts = [] # all perturbation noises (after inverse normalize)

for x, y in test_dataloader:
    x, y = x.to(device), y.to(device)
    x.requires_grad = True
    y_pred = net(x)

    loss = loss_fn(y_pred, y)
    optimizer.zero_grad()
    loss.backward()

    # TODO: generating adversarial examples with noise 0.01
    x_grad = x.grad
    x_adv, pert = fgsm_attack(x,x_grad, eps-0.01)

    x_advs.append(inv_normalize(x_adv))
    perts.append(inv_normalize(pert))

    # TODO: get predictions of adversarial examples
    y_pred_adv = net(x_adv)

    ys.append(y.detach().cpu().data.numpy())
    y_preds.append(y_pred.detach().cpu().data.numpy())
    y_adv_preds.append(y_pred_adv.detach().cpu().data.numpy())

In [ ]:
# TODO: CONCATENATE all labels and predictions
ys = np.concatenate(ys)
y_preds = np.concatenate(y_preds, axis-0).argmax(axis-1)
y_adv_preds = np.concatenate(y_adv_preds, axis-0).argmax(axis-1)

In [ ]:
print(classification_report(ys, y_preds))

In [ ]:
print(classification_report(ys, y_adv_preds))

---

In [ ]:
net.eval()

In [ ]:
# TODO: CONCATENATE all adversarial examples (x_advs) and perturbation noises (perts)
x_advs =torch.concatenate(x_advs,axis=0)
perts = torch.concatenate(perts, axis=0)

In [ ]:
x_advs.size()

In [ ]:
i = np.random.choice(np.where(y_adv_preds != y_preds)[0])

In [ ]:
print(y_preds[i], y_adv_preds[i])

In [ ]:
# visualize the transformed example
grid = torchvision.utils.make_grid(test_dataset[i][0], nrow=1)
transforms.ToPILImage()(grid)

In [ ]:
# visualize the original
grid = torchvision.utils.make_grid(inv_normalize(test_dataset[i][0]), nrow=1)
transforms.ToPILImage()(grid)

In [ ]:
org_pred = net(test_dataset[i][0].unsqueeze(0).to(device)).max(1).indices[0].item()
print("Original Prediction:", org_pred)

In [ ]:
# TODO: visualize noise perts[i]
# visualize the original
grid = torchvision.utils.make_grid(perts[i][0], nrow=1)
transforms.ToPILImage()(grid)

In [ ]:
# TODO: visualize adversarial examples
# visualize the original
grid = torchvision.utils.make_grid(x_advs[i][0], nrow=1)
transforms.ToPILImage()(grid)

In [ ]:
transformed = transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])(x_advs[i].unsqueeze(0)).to(device)
perb_pred = net(transformed).max(1).indices[0].item()
print("Perturbation Prediction:", perb_pred)